## 1. Загрузка данных и изучение общей информации. 

In [1]:
import pandas as pd
data =pd.read_csv('/datasets/data.csv')
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

1. Согласно приведенному описанию данных и полученной информации о таблице видим:
children — количество детей в семье. Тип данных соответствует данным
days_employed — общий трудовой стаж в днях. В столбце имеются отрицательные значения, чего быть не может и значения после запятой (тип float64), а должны быть целочисленными 
dob_years — возраст клиента в годах. Тип данных соответствует данным
education — уровень образования клиента. Тип данных соответствует данным
education_id — идентификатор уровня образования. Тип данных соответствует данным
family_status — семейное положение. Тип данных соответствует данным
family_status_id — идентификатор семейного положения. Тип данных соответствует данным, но можно привести к булевому значению для экономии пространства, пока этого делать не будем.
gender — пол клиента. Тип данных соответствует данным
income_type — тип занятости. Тип данных соответствует данным
debt — имел ли задолженность по возврату кредитов. Тип данных соответствует данным, но можно привести к булевому значению для экономии пространства, пока этого делать не будем.
total_income — ежемесячный доход. Тип данных соответствует данным
purpose — цель получения кредита. Тип данных соответствует данным

2. В Столбце "children" есть выбросы, такие как -1 и 20

## 2. Предобработка данных

### 2.1. Обработка пропусков

In [10]:
#Определим пропуски
print(data.isnull().sum())
print()
cnt_null = data[(data['days_employed'].isnull()) & (data['total_income'].isnull())]
print(len(cnt_null))
print()
print(data['income_type'].value_counts())
print()

#Что не так с кол-вом детей?
#data['children'].value_counts()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

2174
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64



In [ ]:
data['days_employed'] = data['days_employed'].fillna('0')
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.isnull().sum()

#### Вывод

В столбце days_employed(общий трудовой стаж) и total_income(ежемесячный доход) имеется 2174 пропуска и это те же строки.
Пропуски в days_employed и total_income могут означать отсутствие стажа и соответственно отсутствие дохода,но прямой связи с типом занятости нет.

Т.к. данные в столбце days_employed не участвуют в формировании ответа на поставленный вопрос заменим пропущенненные значения 0.
Значения в total_income потребуются для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" заменим пропуски на медиану

### 2.2. Замена типа данных

In [ ]:
print(data.info())

# Приводем столбец days_employed к int и избавляемся от отрицательных значений 
data['days_employed'] = abs(data['days_employed'].astype('int'))

# Приводем столбец total_income к float64
data['total_income'] = pd.to_numeric(data['total_income'], errors='coerce') 
print(data.info())
data.head()

#### Вывод

После замены пропущенных значений тип столбцов days_employed и total_income изменился на object. Вернем тип данных float64 столбцу total_income и days_employed, а так же избавимся от отрицательных значений в days_employed

### 2.3. Обработка дубликатов

In [ ]:
print('Дубликатов в таблице:', data.duplicated().sum())
# Перевод значений в столбцах формата str к нижнему регистру
columns_str =['education', 'family_status', 'gender', 'income_type', 'purpose']
for row in columns_str:
    data[row] = data[row].str.lower()
data.head()
print('Дубликатов в таблице после приведения:', data.duplicated().sum())

# Удаляем дубликаты
data = data.drop_duplicates()
#Удаляем появившиеся пропуски и сбрасываем индексы
data = data.dropna().reset_index(drop = True)
print('Дубликатов в таблице после удаления:', data.duplicated().sum())

#### Вывод

1. В таблице 54 набора строк имеющих дубликаты
2. После перевода регистра количество обнаруженных строк с дубликатами увеличилось до 71
Причины появления дубликатов: Однозначно ответить на этот вопрос без наличия столбца определяющего уникальность (например ФИО заемщика или его ID в системе) нельзя, по этому считаем, что все выявленные дубли являются арифактами и результатом некоректной выгрузки или внесения данных и подлежат удалению.

### 2.4. Лемматизация

In [ ]:
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter 


lemmas = []    
for row in data['purpose']:
    lemmas += m.lemmatize(row)  
Counter(lemmas)



#### Вывод

По результату лематизации видно, что цели кредита можно разделить на несколько категорий, а именно: недвижимость, образование, автомобиль и свадьба

### 2.5. Категоризация данных

In [ ]:
lemass2 =  data['purpose'].apply(m.lemmatize)
# Функция определения категории цели займа:
def purpose_category(lemm):
    if ('жилье' in lemm) or  ('недвижимость' in lemm):
        return 'недвижимости'
    if 'образование' in lemm:
        return 'образование'
    if 'автомобиль' in lemm:
        return 'автомобиль'
    if 'свадьба' in lemm:
        return 'свадьба'
        
data['purpose_category'] = lemass2.apply(purpose_category) # Добавляем столбец с категорией цели займа
print('1.Колво заемщиков по категории займа')
print(data['purpose_category'].value_counts())
print('--------')

# Функция определения категории наличия детей:
def children_category(children):
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'
    
data['children_category'] =data['children'].apply(children_category) # Добавляем столбец с категорией наличия детей
print('2.Колво заемщиков по категории наличия детей')
print(data['children_category'].value_counts())
print('--------')

# Определение диапазона значений в размере дохода
print('3.Диапазон значений в размере дохода')
print('Минимальный уровень дохода:', data['total_income'].min())
print('Максимальный уровень дохода:', data['total_income'].max())
print('Средний уровень дохода:', data['total_income'].mean())
print('Медиана уровеня дохода:', data['total_income'].median())
print('нижний порог среднего:', ((data['total_income'].median()+ data['total_income'].min())/2))
print('верхний порог среднего:', ((data['total_income'].median()+ data['total_income'].max())/2))
print('--------')

# Фукция назначения категории размера дохода:
def total_income_category(income):
    if income <= 82842:
        return 'низкий уровень дохода'
    if income > 1205311:
        return 'высокий уровень дохода'
    return 'средний уровень дохода'
data['total_income_category'] =data['total_income'].apply(total_income_category)
print('4.Колво заемщиков по категории уровня дохода')
print(data['total_income_category'].value_counts())
print('--------')

data.head()

#### Вывод

Добавлены категории к данным, которые в дальнейшем будут участвовать в ответах на вопрос.
1. категория займа, упростит создание выборки для ответа на вопрос "Как разные цели кредита влияют на его возврат в срок?"
Для создания данной категории использовался метод лематизации, далее был просмотрен имеющийся набор ключевых слов и на основании этого созданы категории: недвижимость, автомобиль,образование, свадьба
2. Категория наличия/отсутствия детей нужна для ответа на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?"
Все данные разделены на 2 категории, те у кого есть дети и те у кого их нет
3. Категория по уровню дохода нужна для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"
Для определения категорий проверили диапазон значений -минимальную и максимальную зарплату в выборке, а так же на основании среднего и медианы определили примерный диапазон для разделения. Минимальный порог для средней категории- это ~ середина между медианой и минимальным значением, аналогично определен верхний порог.

## 3. Ответьте на вопросы

### 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data_pivot = data.pivot_table(index=['children_category'], columns='debt', values='gender', aggfunc='count')
data_pivot['procentil'] = data_pivot[1] / data_pivot[0] *100
print(data_pivot.sort_values(by = 'procentil', ascending = True))

data_pivot2 = data.pivot_table(index=['children'], columns='debt', values='gender', aggfunc='count')
data_pivot2['procentil'] = data_pivot2[1] / data_pivot2[0] *100
data_pivot2.sort_values(by = 'procentil', ascending = True)

#### Вывод

Процент заемщиков с детьми и задолженостью по кредиту = 10%,а заемциков без детей с задолженостью - 8%. Похоже, что наличие детей увеличивает вероятность задолжнености.
Но если более детально посмотреть на данные и попытатся выяснить влияет ли кол-во детей на вероятность задолености видно, что прямой корелляции нет.
Соответственно нельзя утверждать, что наличие детей напрямую влияет на наличие задолжености

### 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot_family_status = data.pivot_table(index=['family_status'], columns='debt', values='gender', aggfunc='count')
data_pivot_family_status['procentil'] = data_pivot_family_status[1] / data_pivot_family_status[0] *100
data_pivot_family_status.sort_values(by = 'procentil', ascending = True)

#### Вывод

Люди не имевшие официально оформленого брака имеют задолженность по кредиту чаще, чем вдовцы,разведенные или состоящие в браке на текущий момент

### 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_pivot_total_income = data.pivot_table(index=['total_income_category'], columns='debt', values='gender', aggfunc='count')
data_pivot_total_income['procentil'] = data_pivot_total_income[1] / data_pivot_total_income[0] *100
data_pivot_total_income.sort_values(by = 'procentil', ascending = True)

#### Вывод

Уровень дохода не влияет на вероятность просрочки

### 3.4. Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot_purpose = data.pivot_table(index=['purpose_category'], columns='debt', values='gender', aggfunc='count')
data_pivot_purpose['procentil'] = data_pivot_purpose[1] / data_pivot_purpose[0] *100
data_pivot_purpose.sort_values(by = 'procentil', ascending = True)

#### Вывод

В случае займа на образование и автомобиль вероятность просрочки по выплатам увеличивается.

## Общий вывод

На задолженость по займу в большей степени влияет семейное положение и цель займа.
Уровень дохода и наличие детей не влияет на вероятность просрочки